# The Battle of the Neighborhoods

# Introduction & Business Problem :

# Problem Background:

The City of New York, is the most populous city in the United States. It is diverse and is the financial capital of USA. It is multicultural. It provides lot of business oppourtunities and business friendly environment. It has attracted many different players into the market. It is a global hub of business and commerce. The city is a major center for banking and finance, retailing, world trade, transportation, tourism, real estate, new media, traditional media, advertising, legal services, accountancy, insurance, theater, fashion, and the arts in the United States.

This also means that the market is highly competitive. As it is highly developed city so cost of doing business is also one of the highest. Thus, any new business venture or expansion needs to be analysed carefully. The insights derived from analysis will give good understanding of the business environment which help in strategically targeting the market. This will help in reduction of risk. And the Return on Investment will be reasonable.

# Problem Description:

A restaurant is a business which prepares and serves food and drink to customers in return for money, either paid before the meal, after the meal, or with an open account. The City of New York is famous for its excelllent cuisine. It's food culture includes an array of international cuisines influenced by the city's immigrant history.

In this project I want to find the answers of the below stated questions which is based on Indian Cuisines and Indian Restaurants :

List all great Indian Restaurants that are currently operational in New York City

1.Which are the best localities with great Indian food

2.Which areas lack Indian restaurants

3.What are the top Indian Restaurants in New York

# Target Audience

The target audience who will be benifited by this project can be divided into several groups :

1.Foody people who wants to taste great Indian cuisine in New York city.


# Success Criteria

Success criteria for this project is as follows :

1.Suggesting best Indian restaurant in New York city to the food loving people craving for Indian cuisines.


# Data Set :

For this project I will be using the following datasets :

1.New York City data that contains list Boroughs, Neighbourhoods along with their latitude and longitude.

A. Data source : https://cocl.us/new_york_dataset

B. Description: This data set contains the required information. And we will use this data set to explore various neighbourhoods of New York City

1.Indian restaurants in each neighbourhood of New York City.

A. Data source : Foursquare API

B. Description: By using this API we will get all the venues in each neighbourhood. We can filter these venues to get only Indian restaurants.

1.GeoSpace data

A. Data source : https://data.cityofnewyork.us/City-Government/BoroughBoundaries/tqmj-j8zm

B. Description: By using this geo space data we will get the New York Borough boundaries that will help us visualize choropleth map.

# Methodology

1. We begin with loading New York data from https://cocl.us/new_york_dataset
2. Then we preprocessed and created dataframe from the data with columns Borough, Neighborhood, Lattitude, Longitude
3. After that using the FourSquare API we get the details of all the Indian Restaurant present in all the neighborhoods and store them.
4. We get the neighborhoods which do not have any Indian Restaurants.
5. We then get the details such as Ratings, Price, Like of each Indian restaurant using FourSquare API from the venue.
6. We then sort the restaurant data based on Rating to get the highest rated Restaurant in New York.
7. We get the average rating of the restaurant of each neighborhood, which gives the best location to have Indian cuisine.

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
!pip install geocoder
import geocoder
import os
!pip install folium
import folium
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline
print("All Pkg installed")

     |████████████████████████████████| 102kB 10.2MB/s ta 0:00:01
     |████████████████████████████████| 102kB 7.4MB/s ta 0:00:011
All Pkg installed


Import the JSON file from the URL

In [2]:
URL='https://cocl.us/new_york_dataset'
data=requests.get(URL)
Json_data=data.json()
Json_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

Dataframe creation... Step 1 importing  features..output is a list

In [3]:
Json_data_features=Json_data['features']
Json_data_features

[{'type': 'Feature',
  'id': 'nyu_2451_34572.1',
  'geometry': {'type': 'Point',
   'coordinates': [-73.84720052054902, 40.89470517661]},
  'geometry_name': 'geom',
  'properties': {'name': 'Wakefield',
   'stacked': 1,
   'annoline1': 'Wakefield',
   'annoline2': None,
   'annoline3': None,
   'annoangle': 0.0,
   'borough': 'Bronx',
   'bbox': [-73.84720052054902,
    40.89470517661,
    -73.84720052054902,
    40.89470517661]}},
 {'type': 'Feature',
  'id': 'nyu_2451_34572.2',
  'geometry': {'type': 'Point',
   'coordinates': [-73.82993910812398, 40.87429419303012]},
  'geometry_name': 'geom',
  'properties': {'name': 'Co-op City',
   'stacked': 2,
   'annoline1': 'Co-op',
   'annoline2': 'City',
   'annoline3': None,
   'annoangle': 0.0,
   'borough': 'Bronx',
   'bbox': [-73.82993910812398,
    40.87429419303012,
    -73.82993910812398,
    40.87429419303012]}},
 {'type': 'Feature',
  'id': 'nyu_2451_34572.3',
  'geometry': {'type': 'Point',
   'coordinates': [-73.82780644716412, 

In [4]:
#Json_data_geometry=Json_data_features['geometry']
Length_Features=len(Json_data_features)

Extracting feature from JSON and creating lists for each desired feature

In [5]:
Latitude=[]
Longitude=[]
Name=[]
Borough=[]
for i in range(1,Length_Features+1):
    X=Json_data_features[i-1]
    X_geo=X['geometry']
    X_coordiantes=X_geo['coordinates']
    Latitude.append(X_coordiantes[1])
    Longitude.append(X_coordiantes[0])
    Name.append(X['properties']['name'])
    Borough.append(X['properties']['borough'])


In [7]:
NY_df = pd.DataFrame(list(zip(Borough,Name,Latitude,Longitude)), 
               columns =['Borough','Neighborhood','Latitude','Longitude'])
NY_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [7]:
NY_df.shape

(306, 4)

# There are total 306 Neighbourhoods

In [72]:
lat=NY_df.iloc[0,2]
long=NY_df.iloc[0,3]

In [8]:
search_query = 'Restaurant'
radius = 1000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [9]:
CLIENT_ID = '22I4HAWHPCKU4SQWNVOMVDLG2VLQPWZ5CDWLGWZLTYLEFOP0' # your Foursquare ID
CLIENT_SECRET = 'V3R1NJLA4KAK4B041UADGMVXXHFFMIR4OHOYRVXQHPS3FMXG' # your Foursquare Secret
VERSION = '20200531'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 22I4HAWHPCKU4SQWNVOMVDLG2VLQPWZ5CDWLGWZLTYLEFOP0
CLIENT_SECRET:V3R1NJLA4KAK4B041UADGMVXXHFFMIR4OHOYRVXQHPS3FMXG


In [100]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, 40.61100890202044,-73.99517998380729, VERSION, search_query, radius, LIMIT)

url

'https://api.foursquare.com/v2/venues/search?client_id=VHHOWATJ542UNDLKMWZXDCD1KM4XNQKLDKKH2X03G02H4SXK&client_secret=OT2BBQA0BWP5P5ULVPRIWJS4H5O5WFSAR02FLX0XPIGEL0ZN&ll=40.61100890202044,-73.99517998380729&v=20200530&query=Restaurant&radius=1000&limit=30'

In [105]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed2b829211536001baab78f'},
 'response': {'venues': [{'id': '50ae2b07e4b063e34134a682',
    'name': 'HoWong Restaurant 好旺',
    'location': {'address': '6702 Bay Pkwy',
     'crossStreet': 'at 68th St',
     'lat': 40.61239259825548,
     'lng': -73.9831551557067,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.61239259825548,
       'lng': -73.9831551557067}],
     'distance': 1027,
     'postalCode': '11204',
     'cc': 'US',
     'city': 'Brooklyn',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['6702 Bay Pkwy (at 68th St)',
      'Brooklyn, NY 11204',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d145941735',
      'name': 'Chinese Restaurant',
      'pluralName': 'Chinese Restaurants',
      'shortName': 'Chinese',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1590868348',
   

In [108]:
shortname = results['response']['venues'][0]['name']

In [109]:
shortname

'HoWong Restaurant 好旺'

Function definition for getting venue list from a particular neighborhood

In [10]:
def get_venue_list(lat,long):
    lattitude = lat
    longitude = long
    #print("Lat is",lattitude,"Long is",longitude)
    df=pd.DataFrame(columns=['Name','Address','ID'])
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lattitude, longitude, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    results
    results_venue = results['response']['venues']
    length =len(results_venue)
    #print(length)
    for i in range(1,length+1):
        #print(i)
        try:
            shortname=results_venue[i-1]['categories'][0]['shortName']
            #print(shortname)
            if(results_venue[i-1]['categories'][0]['shortName']=='Indian'):
                #print("inside if")
                Name = results_venue[i-1]['name']
                Address = results_venue[i-1]['location']['formattedAddress'][0]
                ID = results_venue[i-1]['id']
                df=df.append({'Name':Name,'Address':Address,'ID':ID},ignore_index=True)
        except (IndexError,KeyError):
            #print(i)
            pass
    #print(df)
    return df

In [133]:
#result=get_venue_list(40.61100890202044,-73.99517998380729)
#result=get_venue_list(40.866858107252696,-73.83579759808117)

In [62]:
result.head()

,Name,Address


For each neighbourhood information for venues are collected and the Indian Restaurants are separated.

In [11]:
row_number = NY_df.shape[0]
Venue_df = pd.DataFrame(columns=['Neighborhood','Restaurant Name','Restaurant Address','ID'])
for rows in range(1,row_number+1):
    lat=NY_df.iloc[rows-1,2]
    long=NY_df.iloc[rows-1,3]
    neighborhood = NY_df.iloc[rows-1,1]
    result = get_venue_list(lat,long)
    if(result.empty):
        print("No result came")
    else:
        for r1 in range(1,result.shape[0]+1):
            name = result.iloc[r1-1,0]
            address = result.iloc[r1-1,1]
            res_id = result.iloc[r1-1,2]
            Venue_df=Venue_df.append({'Neighborhood':neighborhood,'Restaurant Name':name,'Restaurant Address':address,'ID':res_id},ignore_index=True)

Venue_df.head()

No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result came
No result 

,Neighborhood,Restaurant Name,Restaurant Address,ID
0,Kingsbridge,Tazmohol Indian Restaurant,156 W 231st St,4e4ddf74bd4101d0d79d3e42
1,Marble Hill,Tazmohol Indian Restaurant,156 W 231st St,4e4ddf74bd4101d0d79d3e42
2,Norwood,Aman Restaurant,"Bronx, NY",57363fd4498eb703a5b5ea35
3,Soundview,Bengal Restaurant,1209 White Plains Rd,4f3ef014e4b020913ba39e9b
4,Parkchester,Premium Sweets & Restaurant,2104 Starling Ave,55dfa36a498e164ef19bef7b


In [12]:
Venue_df.shape

(128, 4)

In [66]:
Venue_Indian = NY_df.merge(Venue_df,on='Neighborhood',how='left')
l=Venue_Indian[Venue_Indian['Borough']=='Bronx']
Venue_Indian

,Borough,Neighborhood,Latitude,Longitude,Restaurant Name,Restaurant Address,ID
0,Bronx,Wakefield,40.894705,-73.847201,NaN,NaN,NaN
1,Bronx,Co-op City,40.874294,-73.829939,NaN,NaN,NaN
2,Bronx,Eastchester,40.887556,-73.827806,NaN,NaN,NaN
3,Bronx,Fieldston,40.895437,-73.905643,NaN,NaN,NaN
4,Bronx,Riverdale,40.890834,-73.912585,NaN,NaN,NaN
5,Bronx,Kingsbridge,40.881687,-73.902818,Tazmohol Indian Restaurant,156 W 231st St,4e4ddf74bd4101d0d79d3e42
6,Manhattan,Marble Hill,40.876551,-73.910660,Tazmohol Indian Restaurant,156 W 231st St,4e4ddf74bd4101d0d79d3e42
7,Bronx,Woodlawn,40.898273,-73.867315,NaN,NaN,NaN
8,Bronx,Norwood,40.877224,-73.879391,Aman Restaurant,"Bronx, NY",57363fd4498eb703a5b5ea35
9,Bronx,Williamsbridge,40.881039,-73.857446,NaN,NaN,NaN


In [53]:
Indian=Venue_Indian.groupby('Borough')['ID'].count()

# Which Borough has Highest number of Indian restaurants

In [54]:
Indian_df=Indian.to_frame()
Indian_df
#print(Indian_df[Indian_df.ID == Indian_df.ID.max()])

,ID
Borough,
Bronx,7
Brooklyn,28
Manhattan,17
Queens,71
Staten Island,8


# Neighbourhoods having no Indian Restaurants

In [78]:
Indian=Venue_Indian.groupby('Neighborhood')['ID'].count().to_frame()
No_Indian=Indian[Indian['ID']==0]
No_Indian

,ID
Neighborhood,
Allerton,0
Annadale,0
Arden Heights,0
Arlington,0
Arrochar,0
Arverne,0
Astoria,0
Auburndale,0
Bath Beach,0


# Which Neighborhood has highest number of Indian Restaurant

In [17]:
Indian_neigh=Venue_Indian.groupby('Neighborhood')['ID'].count()

In [18]:
Indian_neigh_df=Indian_neigh.to_frame()
print(Indian_neigh_df[Indian_neigh_df.ID == Indian_neigh_df.ID.max()])

                 ID
Neighborhood       
Jamaica Estates   6


# Droping all the rows with NaN values

In [19]:
Indian_rest_df=Venue_Indian
Indian_rest_df.head()

,Borough,Neighborhood,Latitude,Longitude,Restaurant Name,Restaurant Address,ID
0,Bronx,Wakefield,40.894705,-73.847201,NaN,NaN,NaN
1,Bronx,Co-op City,40.874294,-73.829939,NaN,NaN,NaN
2,Bronx,Eastchester,40.887556,-73.827806,NaN,NaN,NaN
3,Bronx,Fieldston,40.895437,-73.905643,NaN,NaN,NaN
4,Bronx,Riverdale,40.890834,-73.912585,NaN,NaN,NaN


In [20]:
Indian_rest_df.dropna(inplace=True)
Indian_rest_df.head()

,Borough,Neighborhood,Latitude,Longitude,Restaurant Name,Restaurant Address,ID
5,Bronx,Kingsbridge,40.881687,-73.902818,Tazmohol Indian Restaurant,156 W 231st St,4e4ddf74bd4101d0d79d3e42
6,Manhattan,Marble Hill,40.876551,-73.910660,Tazmohol Indian Restaurant,156 W 231st St,4e4ddf74bd4101d0d79d3e42
8,Bronx,Norwood,40.877224,-73.879391,Aman Restaurant,"Bronx, NY",57363fd4498eb703a5b5ea35
26,Bronx,Soundview,40.821012,-73.865746,Bengal Restaurant,1209 White Plains Rd,4f3ef014e4b020913ba39e9b
30,Bronx,Parkchester,40.837938,-73.856003,Premium Sweets & Restaurant,2104 Starling Ave,55dfa36a498e164ef19bef7b


Testing of Sample Venues

In [26]:
venue_id = '3fd66200f964a520f4e41ee3' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/3fd66200f964a520f4e41ee3?client_id=22I4HAWHPCKU4SQWNVOMVDLG2VLQPWZ5CDWLGWZLTYLEFOP0&client_secret=V3R1NJLA4KAK4B041UADGMVXXHFFMIR4OHOYRVXQHPS3FMXG&v=20200531'

In [79]:
#result = requests.get(url).json()
#result

Function for getting Venue details by providing ID of the venue

In [33]:
def get_venue_ratings(ID):
    venue_id = ID
    result_list=[]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    try:
        
        ratings=result['response']['venue']['rating']
        Likes=result['response']['venue']['likes']['count']
        price=result['response']['venue']['price']['message']
        result_list.append(ratings)
        result_list.append(Likes)
        result_list.append(price)
    except:
        pass
    return result_list

Creating dataframe of Restaurants with Ratings

In [37]:
Restaurant_rating = pd.DataFrame(columns=['ID','Name','Rating','Like','Price'])
for i in range(1,Indian_rest_df.shape[0]+1):
    id=Indian_rest_df.iloc[i-1,6]
    name=Indian_rest_df.iloc[i-1,4]
    temp=get_venue_ratings(id)
    try:
        rating=temp[0]
        like=temp[1]
        price=temp[2]
        Restaurant_rating=Restaurant_rating.append({'ID':id,'Name':name,'Rating':rating,'Like':like,'Price':price},ignore_index=True)
    except:
        pass
Restaurant_rating.head()

,ID,Name,Rating,Like,Price
0,4af0d31bf964a5207ddf21e3,Pak Nasheman,7.5,9,Moderate
1,4b718914f964a520c04b2de3,Madina Restaurant and Sweets,6.7,17,Moderate
2,4b718914f964a520c04b2de3,Madina Restaurant and Sweets,6.7,17,Moderate
3,4c673e729cb82d7f024593d2,Sapid Indian Restaurant,6.0,11,Moderate
4,52f18573498ec2c34e830ffd,Kanan's Indian Restaurant,7.9,24,Moderate


Sorting the restaurants on basis of Rating

In [41]:
Restaurant_rating.sort_values(by=['Rating'],ascending=False,inplace=True)

In [42]:
Restaurant_rating.head()

,ID,Name,Rating,Like,Price
7,4ae7876ef964a5201eac21e3,Kismat Indian Restaurant,8.0,45,Moderate
4,52f18573498ec2c34e830ffd,Kanan's Indian Restaurant,7.9,24,Moderate
5,52f18573498ec2c34e830ffd,Kanan's Indian Restaurant,7.9,24,Moderate
8,54c2bd96498eaf5142e3fe92,Clove Indian Restaurant & Bar,7.8,32,Moderate
0,4af0d31bf964a5207ddf21e3,Pak Nasheman,7.5,9,Moderate


# The highest rated Indian Restaurant in New York

In [43]:
print("Restaurant with highest rating is",Restaurant_rating.iloc[0,1],'with rating =',Restaurant_rating.iloc[0,2])

Restaurant with highest rating is Kismat Indian Restaurant with rating = 8.0


In [44]:
Indian_rest_rating = Indian_rest_df.merge(Restaurant_rating,on='ID',how='left')
Indian_rest_rating.head()

,Borough,Neighborhood,Latitude,Longitude,Restaurant Name,Restaurant Address,ID,Name,Rating,Like,Price
0,Bronx,Kingsbridge,40.881687,-73.902818,Tazmohol Indian Restaurant,156 W 231st St,4e4ddf74bd4101d0d79d3e42,NaN,NaN,NaN,NaN
1,Manhattan,Marble Hill,40.876551,-73.910660,Tazmohol Indian Restaurant,156 W 231st St,4e4ddf74bd4101d0d79d3e42,NaN,NaN,NaN,NaN
2,Bronx,Norwood,40.877224,-73.879391,Aman Restaurant,"Bronx, NY",57363fd4498eb703a5b5ea35,NaN,NaN,NaN,NaN
3,Bronx,Soundview,40.821012,-73.865746,Bengal Restaurant,1209 White Plains Rd,4f3ef014e4b020913ba39e9b,NaN,NaN,NaN,NaN
4,Bronx,Parkchester,40.837938,-73.856003,Premium Sweets & Restaurant,2104 Starling Ave,55dfa36a498e164ef19bef7b,NaN,NaN,NaN,NaN


In [45]:
Indian_rest_rating.dropna(inplace=True)
Indian_rest_rating

,Borough,Neighborhood,Latitude,Longitude,Restaurant Name,Restaurant Address,ID,Name,Rating,Like,Price
9,Brooklyn,Brighton Beach,40.576825,-73.965094,Pak Nasheman,3149 Coney Island Ave,4af0d31bf964a5207ddf21e3,Pak Nasheman,7.5,9,Moderate
11,Brooklyn,Flatbush,40.636326,-73.958401,Madina Restaurant and Sweets,563 Coney Island Ave (Beverly Rd & Lewis Pl),4b718914f964a520c04b2de3,Madina Restaurant and Sweets,6.7,17,Moderate
12,Brooklyn,Flatbush,40.636326,-73.958401,Madina Restaurant and Sweets,563 Coney Island Ave (Beverly Rd & Lewis Pl),4b718914f964a520c04b2de3,Madina Restaurant and Sweets,6.7,17,Moderate
13,Brooklyn,Flatbush,40.636326,-73.958401,Madina Restaurant and Sweets,563 Coney Island Ave (Beverly Rd & Lewis Pl),4b718914f964a520c04b2de3,Madina Restaurant and Sweets,6.7,17,Moderate
16,Brooklyn,Kensington,40.642382,-73.980421,Madina Restaurant and Sweets,563 Coney Island Ave (Beverly Rd & Lewis Pl),4b718914f964a520c04b2de3,Madina Restaurant and Sweets,6.7,17,Moderate
17,Brooklyn,Kensington,40.642382,-73.980421,Madina Restaurant and Sweets,563 Coney Island Ave (Beverly Rd & Lewis Pl),4b718914f964a520c04b2de3,Madina Restaurant and Sweets,6.7,17,Moderate
18,Brooklyn,Kensington,40.642382,-73.980421,Madina Restaurant and Sweets,563 Coney Island Ave (Beverly Rd & Lewis Pl),4b718914f964a520c04b2de3,Madina Restaurant and Sweets,6.7,17,Moderate
20,Brooklyn,Prospect Heights,40.676822,-73.964859,Sapid Indian Restaurant,595 Washington Ave (btw Pacific & Dean),4c673e729cb82d7f024593d2,Sapid Indian Restaurant,6.0,11,Moderate
25,Brooklyn,Carroll Gardens,40.680540,-73.994654,Kanan's Indian Restaurant,452 3rd Ave (9th Street),52f18573498ec2c34e830ffd,Kanan's Indian Restaurant,7.9,24,Moderate
26,Brooklyn,Carroll Gardens,40.680540,-73.994654,Kanan's Indian Restaurant,452 3rd Ave (9th Street),52f18573498ec2c34e830ffd,Kanan's Indian Restaurant,7.9,24,Moderate


In [46]:
temp=Indian_rest_rating
temp.drop(axis=1,columns=['Restaurant Address','Name'],inplace=True)
temp

,Borough,Neighborhood,Latitude,Longitude,Restaurant Name,ID,Rating,Like,Price
9,Brooklyn,Brighton Beach,40.576825,-73.965094,Pak Nasheman,4af0d31bf964a5207ddf21e3,7.5,9,Moderate
11,Brooklyn,Flatbush,40.636326,-73.958401,Madina Restaurant and Sweets,4b718914f964a520c04b2de3,6.7,17,Moderate
12,Brooklyn,Flatbush,40.636326,-73.958401,Madina Restaurant and Sweets,4b718914f964a520c04b2de3,6.7,17,Moderate
13,Brooklyn,Flatbush,40.636326,-73.958401,Madina Restaurant and Sweets,4b718914f964a520c04b2de3,6.7,17,Moderate
16,Brooklyn,Kensington,40.642382,-73.980421,Madina Restaurant and Sweets,4b718914f964a520c04b2de3,6.7,17,Moderate
17,Brooklyn,Kensington,40.642382,-73.980421,Madina Restaurant and Sweets,4b718914f964a520c04b2de3,6.7,17,Moderate
18,Brooklyn,Kensington,40.642382,-73.980421,Madina Restaurant and Sweets,4b718914f964a520c04b2de3,6.7,17,Moderate
20,Brooklyn,Prospect Heights,40.676822,-73.964859,Sapid Indian Restaurant,4c673e729cb82d7f024593d2,6.0,11,Moderate
25,Brooklyn,Carroll Gardens,40.680540,-73.994654,Kanan's Indian Restaurant,52f18573498ec2c34e830ffd,7.9,24,Moderate
26,Brooklyn,Carroll Gardens,40.680540,-73.994654,Kanan's Indian Restaurant,52f18573498ec2c34e830ffd,7.9,24,Moderate


In [47]:
temp.drop_duplicates(inplace=True)
temp

,Borough,Neighborhood,Latitude,Longitude,Restaurant Name,ID,Rating,Like,Price
9,Brooklyn,Brighton Beach,40.576825,-73.965094,Pak Nasheman,4af0d31bf964a5207ddf21e3,7.5,9,Moderate
11,Brooklyn,Flatbush,40.636326,-73.958401,Madina Restaurant and Sweets,4b718914f964a520c04b2de3,6.7,17,Moderate
16,Brooklyn,Kensington,40.642382,-73.980421,Madina Restaurant and Sweets,4b718914f964a520c04b2de3,6.7,17,Moderate
20,Brooklyn,Prospect Heights,40.676822,-73.964859,Sapid Indian Restaurant,4c673e729cb82d7f024593d2,6.0,11,Moderate
25,Brooklyn,Carroll Gardens,40.680540,-73.994654,Kanan's Indian Restaurant,52f18573498ec2c34e830ffd,7.9,24,Moderate
27,Brooklyn,Gowanus,40.673931,-73.994441,Kanan's Indian Restaurant,52f18573498ec2c34e830ffd,7.9,24,Moderate
35,Brooklyn,Prospect Park South,40.647009,-73.962613,Madina Restaurant and Sweets,4b718914f964a520c04b2de3,6.7,17,Moderate
39,Manhattan,Washington Heights,40.851903,-73.936900,Kismat Indian Restaurant,4ae7876ef964a5201eac21e3,8.0,45,Moderate
40,Manhattan,Hamilton Heights,40.823604,-73.949688,Clove Indian Restaurant & Bar,54c2bd96498eaf5142e3fe92,7.8,32,Moderate
43,Manhattan,East Harlem,40.792249,-73.944182,Polash Indian restaurant,4a60aa67f964a520f9c01fe3,6.6,19,Moderate


In [50]:
x=temp.groupby('Neighborhood')['Rating'].mean().to_frame()
x

,Rating
Neighborhood,
Brighton Beach,7.5
Carroll Gardens,7.9
Ditmas Park,6.7
East Harlem,6.4
East Village,6.1
Flatbush,6.7
Gowanus,7.9
Hamilton Heights,7.8
Kensington,6.7


In [52]:
Good_rating_Indian_restaurant_neighbourhood=x[x['Rating']>x['Rating'].mean()]
Good_rating_Indian_restaurant_neighbourhood

,Rating
Neighborhood,
Brighton Beach,7.5
Carroll Gardens,7.9
Gowanus,7.9
Hamilton Heights,7.8
Washington Heights,8.0


# Conclusion

From the analysis we are seeing the following conclusion : 

1.Which are the best localities with great Indian food


Brighton Beach, Carroll Gardens, Gowanus, Hamilton Heights, Washington Heights

2.Which areas lack Indian restaurants


Neighbourhoods like Allerton,Annedale etc. Total 226.

3.What is the top Indian Restaurants in New York


Kismat Indian Restaurant in Washington Heights